[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/RonPlusSign/AnomalySegmentation/blob/main/Project6.ipynb)

# **Anomaly Segmentation Project 6**
##*Andrea Delli, Christian Dellisanti, Giorgia Modi*